Atividades
===

Essas atividade são relativas a aulas de extração de atributos e selecão de atributos. 
- Link para video: https://drive.google.com/file/d/13LLQGLt7QsjKshBXgUdJ4RVOT8N252cU/view
- Link para apresentação: https://docs.google.com/presentation/d/1wctFgQe7TSlBEypZbVQqqsfrnpSgOeXPSO-mCJ6jIz4


> Lembre de criar uma `virtualenv` com os `requirements.txt` do repositório.

> Lembre de criar uma `kernel` do jupyter para seus desenvolvimentos

> **MANTENHA** essa arquivo no local padrão do repositório. Detro da pasta **feature_extraction_and_feature_selection**.

> Enviei apenas o seu notebook para correção. Não é necessário enviar nenhum outro arquivo. Siga o padrão `Atividades-<NOME-DO-ALUNO>.ipynb`

# Feature Extraction

## 1. Realizar a extração de atributos utilzando as técnicas Fourier e HOS nas bases de dados

- Dados de falhas em aerogeradores

As orientações abaixo devem ser seguidas antes de iniciar os trabalhos

> Todo os arquivos nomeados com `v000_ ... .csv` são referentes a essa base.

> A classe referente ao dado está representada por uma TAG no nome do arquivo da seguinte forma: `v000_NORMAL_ ... .csv` é o dado referente a classe de funcionamento normal do gerador. A TAG `v000_SC_LI_LVL3_ ... .csv` é a classe referente a falha tipo 1 e continua. Não se preocupem com a ordem de enumeração das classes nesse momento.

> Há 5 arquivos na pasta.

> Utilizar as colunas a `Current_R`, `Current_S`, `Current_T` para realizar extração de atributos. Perceba que cada coluna é referente a uma sinal de corrente elétrica, portanto cada um são representados vetores de dimensões $1x50000$.

> Antes de realizar a extração de atributos deve-se dividir cada sinal em 10 partes de tamanho $1x5000$. A ideia aqui é aumentar o número de amostra da base de dados por 10. Pense que cada parte é um recorte do sinal, e ao serem concatenadas retornarão o sinal original.

# Imports  

### Importing feature extraction module  

In [1]:
!git clone https://github.com/navarmn/feature_extraction_signal.git

fatal: destination path 'feature_extraction_signal' already exists and is not an empty directory.


In [2]:
from feature_extraction_signal.src import feature_extraction

### Importing pandas-profiling

In [3]:
from pandas_profiling import ProfileReport

### importing pandas and others tools

In [4]:
import pandas as pd
import numpy as np
import os.path
from matplotlib import pyplot as plt

### Carregue a base

In [5]:
dataset_folder = os.path.join('.', 'datasets')
DATAFILES = [
    'v000_FAULT_SC_HI_LVL2_FR6000_FG5942_L000_0,6IN_SENSORC.csv',
    'v000_FAULT_SC_LI_LVL3_FR4500_FG4365_L000_0,8IN_SENSORC.csv',
    'v000_FAULT_SC_LI_LVL3_FR6000_FG5927_L000_0,4IN_SENSORC.csv',
    'v000_NORMAL_FR4500_FG4385_L000_1,0IN_SENSORC.csv',
    'v000_NORMAL_FR6000_FG5955_L000_0,5IN_SENSORC.csv'
]

In [6]:
'''
We are going to have an array which each element is a dataframe with the full signal
'''

signals = [pd.read_csv(os.path.join(dataset_folder, DATAFILES[x])).iloc[:, :3] for x in range(len(DATAFILES))]

### Split signals

#### This code below will split the dataframe in an array with N dataframes

In [7]:
def splitDataframe(df, slices=10):
    #collecting number of rows and columns
    df_rows, df_columns = df.shape
    
    #defining the slice
    slice = int(df_rows/slices)
    
    #defining variables to start spliting the dataframe
    prev_slice, current_slice = 0, 0
    slices_array = [] #array that will receive the slices
    for x in range(slices):
        #update the prev and current slice variables and append the slice to slices_array 
        prev_slice = current_slice
        current_slice+=slice
        slices_array.append(df.iloc[prev_slice:current_slice])
    return slices_array    

### Joining a sliced dataframe

In [8]:
def joinDataframe(array_of_dfs):
    dataframe = pd.DataFrame([])
    for df in array_of_dfs:
        dataframe = dataframe.append(df, ignore_index=True)
    return dataframe

In [9]:
'''
Now we are going to split each signal, that by now are dataframes. So at the end of the process we
will have an array which each element it will be an another array with the sliced signals.

[[slice1DF, slice2DF, ..., slice10DF], ...]

'''

splited_signals = []
for signal in signals:
    splited_signals.append(splitDataframe(signal))

    
#selecting just the three fisrt columns of the splited dataframe
#array_of_dfs = [df.iloc[:, :3] for df in array_of_dfs]

### a) Comparativo da média do sinal e sua distribuição

- Realize o calculo da média e desvio padrão em cada um dos sinais e exiba o resultado em um dataframe. Discuta suas conclusões.

**help**: utilize as funcões nativas do numpy ou do pandas. Lembre que o sinal é uma senoide, qual a média em uma senoide simétrica?

In [14]:
def mean_and_std(array_of_dfs, how='slices'):
    '''
    Two types of use:
    full: Use the full signal to calculate the mean and the standard deviation.
    
    slices: Use the slices to calculate the mean and the standard deviation and return the result
    of each slice as a Dataframe
    '''

    #dataframes that will be merged
    df_mean, df_std = pd.DataFrame([]),pd.DataFrame([])
    
    #dataframe that will be returned
    return_df = pd.DataFrame([])
    
    #joins the dataframe
    dataframe = joinDataframe(array_of_dfs)

    #getting columns names and changing them
    cols_names = dataframe.columns
    mean_names, std_names = ['mean_'+name for name in cols_names],['std_'+name for name in cols_names]
    
    if how == 'full':
        
        #calculating the mean and the standard deviation of the dataframe
        serie_mean, serie_std = dataframe.mean(), dataframe.std()
        
        #changing the index of the series
        serie_mean.index,serie_std.index = mean_names,std_names
        
        #returns the mean series concatenated with the std series
        return pd.concat([serie_mean, serie_std], join='outer', sort=False)

    if how == 'slices':
        
        #deleting dataframe variable that will not be used anymore
        del dataframe
        
        for df in array_of_dfs:

            #calculating the mean and the standard deviation of the current dataframe(df)
            serie_mean, serie_std = df.mean(), df.std()

            #changing the index of the series
            serie_mean.index,serie_std.index = mean_names,std_names

            #appending the serie_mean and the serie_std to df_mean and df_std, respectively
            df_mean, df_std = df_mean.append(serie_mean, ignore_index=True), df_std.append(serie_std, ignore_index=True, sort=False)
    
        #Merging the two dataframes (df_mean and df_std)
        return_df = df_mean.merge(df_std, left_index=True, right_index=True, how='outer')
        
            
    return return_df

In [ ]:
for i, splited_signal in enumerate(splited_signals):
    print('\n'+DATAFILES[i]+'\nSignal {}'.format(i))
    m_std = mean_and_std(splited_signal, how='slices')
    display(m_std)
    

##### È possível notar que temos a média do sinal por volta de zero por causa de sua característica periódica.

### b) Extração de atributos no sinal

- Utilize o código de `feature_extraction_signal` em: https://github.com/navarmn/feature_extraction_signal
- Use as classes `Fourier` e `HOS` de maneira adequada. Lembre do método `.transform()`.
- Na classe `Fourier`utilize os parâmetros: `fs=5000`; o valor de fundamental está na TAG do nome em `_FG5955_`significa `fundamental=59.55`. `harmonics=(0.5, 1, 1.5, 2.5, 3, 5, 7)`, mas sintam-se à vontade para buscar outras harmônicas;
- Os rótulos devem serem impostos de acordo com a descrição da tag feita cima. 
- Monte um dataframe que contenham os atributos em cada coluna e a útlima com o rótulo. Concatene os artibutos extraídos da `Current_R`, `Current_S` e `Current_T`.
- Deverá ser feito um dataframe para os atributos de Fourier e um para os atributos de HOS.

### Extracting fundamental frequencies


In [69]:
fund_freq = [] #fundamental frequencies

#simple string manipulation to extract the fundamental frequencies from the datafile's name
for file_name in DATAFILES:
    a = file_name.split('_')
    fr = a[3] if a[1] == 'NORMAL' else a[6] 
    fund_freq.append(float(fr[2:])/100)
print("fund_freq =",fund_freq)

fund_freq = [59.42, 43.65, 59.27, 43.85, 59.55]


### Fourier features

In [71]:
def extractSplittedSignalFourierFeatures(splited_signal, fund_freq, harm=(0.5, 1, 1.5, 2.5, 3, 5, 7)):

    #Fourier feature extractor
    ffe = feature_extraction.Fourier(fundamental=fund_freq, fs=500.0, harmonics=harm)

    #dataframe that will be returned
    retDf = pd.DataFrame([])

    
    for colName in splited_signal[0].columns:
        oneColumnDF = pd.DataFrame([])
        
        #creating columns labels
        currentIndexNames = [colName+"_"+str(harm[x]) for x in range(7)]

        for signalSlice in splited_signal:

            out_fourier = ffe.transform(signalSlice[colName])

            features = pd.Series(np.transpose(out_fourier['features']))  

            features.index = currentIndexNames

            oneColumnDF = oneColumnDF.append(features, ignore_index=True)

        retDf = retDf.merge(oneColumnDF, how='outer', left_index=True,right_index=True)
    return retDf

In [ ]:
for i,splited_signal in enumerate(splited_signals):
    print('\n'+DATAFILES[i]+'\nSignal {}'.format(i)+'\nFourier Features:')
    display(extractSplittedSignalFourierFeatures(splited_signal, fund_freq=fund_freq[i]))

### HOS features

In [ ]:
def extractSplittedSignalHOSFeatures(splited_signal):
    fe_hos = feature_extraction.HOS()
    
    retDf = pd.DataFrame([])
    
    for colName in splited_signal[0].columns:
        oneColumnDF = pd.DataFrame([])
        
        #creating columns labels
        featuresNames = ['rms', 'variance', 'skewness', 'kurtosis']
        currentIndexNames = [colName+"_"+x for x in featuresNames]

        for signalSlice in splited_signal:

            features = fe_hos.transform(signalSlice[colName])

            features = pd.Series(np.transpose(features['features']))  

            features.index = currentIndexNames

            oneColumnDF = oneColumnDF.append(features, ignore_index=True)

        retDf = retDf.merge(oneColumnDF, how='outer', left_index=True,right_index=True)
    return retDf

In [79]:
for i,splited_signal in enumerate(splited_signals):
    print('\n'+DATAFILES[i]+'\nSignal {}'.format(i)+'\nHOS Features:')
    display(extractSplittedSignalHOSFeatures(splited_signal))


v000_FAULT_SC_HI_LVL2_FR6000_FG5942_L000_0,6IN_SENSORC.csv
Signal 0
HOS Features:


,Current_R_kurtosis,Current_R_rms,Current_R_skewness,Current_R_variance,Current_S_kurtosis,Current_S_rms,Current_S_skewness,Current_S_variance,Current_T_kurtosis,Current_T_rms,Current_T_skewness,Current_T_variance
0,-1.446445,1.291684,-0.004277,1.668738,-1.418205,1.211244,0.011664,1.467376,-1.415615,1.248681,0.006801,1.559508
1,-1.408686,1.297518,0.012420,1.683549,-1.400510,1.211815,0.003994,1.468610,-1.431789,1.251374,-0.011335,1.566151
2,-1.423154,1.294843,-0.004000,1.676938,-1.457294,1.204686,-0.002040,1.451276,-1.480777,1.242784,0.012227,1.544766
3,-1.468980,1.294198,0.002751,1.675207,-1.489261,1.208571,0.006711,1.460915,-1.430603,1.251857,0.007442,1.567415
4,-1.465938,1.296551,0.013003,1.681225,-1.426711,1.221662,-0.000697,1.492396,-1.434491,1.248086,0.007001,1.557986
5,-1.436158,1.289001,-0.004802,1.661822,-1.414078,1.209481,0.018315,1.463109,-1.419430,1.245057,0.003501,1.550443
6,-1.406074,1.296967,0.017590,1.682089,-1.402980,1.209359,-0.000979,1.462493,-1.439200,1.245572,-0.001944,1.551740
7,-1.430156,1.296516,-0.008771,1.681285,-1.457447,1.205200,0.005736,1.452774,-1.477752,1.241504,0.011588,1.541624
8,-1.471720,1.296129,0.011852,1.679935,-1.486809,1.211291,0.009285,1.467424,-1.431730,1.261556,-0.006775,1.591735
9,-1.464775,1.298295,-0.002830,1.685867,-1.417600,1.217003,0.008132,1.481183,-1.432403,1.253678,0.008624,1.571981



v000_FAULT_SC_LI_LVL3_FR4500_FG4365_L000_0,8IN_SENSORC.csv
Signal 1
HOS Features:


,Current_R_kurtosis,Current_R_rms,Current_R_skewness,Current_R_variance,Current_S_kurtosis,Current_S_rms,Current_S_skewness,Current_S_variance,Current_T_kurtosis,Current_T_rms,Current_T_skewness,Current_T_variance
0,-1.567843,3.102081,0.009699,9.624446,-1.321806,2.364531,-0.003107,5.592128,-1.393620,2.491024,-0.005918,6.206175
1,-1.508231,3.136758,-0.016235,9.840774,-1.249589,2.401209,0.006532,5.766882,-1.415574,2.458412,-0.003731,6.044816
2,-1.480149,3.173119,0.006277,10.070247,-1.291860,2.442660,-0.019893,5.966891,-1.481063,2.427142,0.003892,5.892142
3,-1.521114,3.175549,0.006790,10.085443,-1.402828,2.464860,-0.010673,6.076592,-1.464560,2.433405,-0.029779,5.922405
4,-1.524411,3.159810,-0.012491,9.986180,-1.376761,2.405895,-0.007735,5.789337,-1.464101,2.490689,0.013724,6.204759
5,-1.568410,3.104530,0.009471,9.639637,-1.314335,2.359404,-0.016952,5.566945,-1.407617,2.496308,0.002898,6.232635
6,-1.520480,3.143901,-0.001617,9.885984,-1.252725,2.402504,0.007149,5.773174,-1.409165,2.452822,-0.016447,6.017037
7,-1.473721,3.159259,-0.010733,9.982566,-1.296462,2.441309,-0.003664,5.961182,-1.474950,2.423010,0.002267,5.872004
8,-1.521273,3.172790,0.004736,10.067849,-1.392716,2.458840,-0.031913,6.045573,-1.474011,2.438444,-0.009923,5.947115
9,-1.530693,3.160054,0.007875,9.987711,-1.371630,2.403601,-0.006488,5.778144,-1.461249,2.488284,-0.013002,6.192159



v000_FAULT_SC_LI_LVL3_FR6000_FG5927_L000_0,4IN_SENSORC.csv
Signal 2
HOS Features:


,Current_R_kurtosis,Current_R_rms,Current_R_skewness,Current_R_variance,Current_S_kurtosis,Current_S_rms,Current_S_skewness,Current_S_variance,Current_T_kurtosis,Current_T_rms,Current_T_skewness,Current_T_variance
0,-1.521887,1.556996,0.008073,2.424641,-1.361106,1.293124,0.007866,1.672421,-1.414539,1.119778,-0.018191,1.254151
1,-1.489971,1.548700,0.012431,2.398374,-1.339144,1.285855,0.001877,1.653404,-1.385794,1.106261,-0.005429,1.224049
2,-1.441026,1.568151,-0.005479,2.459571,-1.301946,1.296422,0.013278,1.680866,-1.404029,1.103983,0.005962,1.219001
3,-1.449412,1.567055,-0.011262,2.456117,-1.354759,1.305302,0.015164,1.704126,-1.443987,1.099057,-0.004336,1.208155
4,-1.515697,1.561223,0.002119,2.437553,-1.430261,1.290347,0.000893,1.665083,-1.378006,1.129521,-0.005244,1.276013
5,-1.521882,1.550004,0.008760,2.402969,-1.411446,1.284153,-0.009843,1.649062,-1.413429,1.114167,0.005444,1.241590
6,-1.492281,1.558064,0.000547,2.428000,-1.341936,1.287703,0.017378,1.658412,-1.385788,1.106932,0.003615,1.225527
7,-1.438976,1.566864,-0.003324,2.455527,-1.301154,1.298235,0.020521,1.685686,-1.400915,1.103368,-0.010143,1.217629
8,-1.453429,1.572847,0.004148,2.473948,-1.348919,1.299558,0.001830,1.688843,-1.444161,1.103906,-0.006638,1.218839
9,-1.510650,1.559136,-0.004696,2.431311,-1.430004,1.293950,-0.001274,1.674380,-1.410541,1.121831,0.008911,1.258754



v000_NORMAL_FR4500_FG4385_L000_1,0IN_SENSORC.csv
Signal 3
HOS Features:


,Current_R_kurtosis,Current_R_rms,Current_R_skewness,Current_R_variance,Current_S_kurtosis,Current_S_rms,Current_S_skewness,Current_S_variance,Current_T_kurtosis,Current_T_rms,Current_T_skewness,Current_T_variance
0,-1.455988,2.919062,-0.006956,8.522613,-1.484746,2.911069,-0.000401,8.476016,-1.475827,2.974108,-0.014169,8.846538
1,-1.473746,2.913977,-0.008870,8.492857,-1.489181,2.902326,-0.002782,8.425178,-1.480733,2.967735,-0.002321,8.809051
2,-1.497808,2.868046,-0.006573,8.227216,-1.499978,2.915474,0.000318,8.500898,-1.423277,2.991036,0.004927,8.947943
3,-1.494225,2.909763,-0.007844,8.468410,-1.456457,2.937234,-0.008976,8.628958,-1.396336,2.999333,-0.002631,8.997724
4,-1.440243,2.913728,0.005102,8.491347,-1.436949,2.931210,-0.013682,8.593172,-1.438391,2.998057,-0.006894,8.990145
5,-1.448949,2.912002,0.000941,8.481121,-1.488744,2.922471,-0.010651,8.542278,-1.478654,2.982705,-0.017708,8.898171
6,-1.480727,2.920185,-0.001387,8.529169,-1.491834,2.905191,-0.004639,8.441783,-1.473433,2.965073,-0.012870,8.793401
7,-1.499681,2.879052,-0.005639,8.290300,-1.491246,2.907836,0.003707,8.457063,-1.436169,2.993104,-0.003990,8.960441
8,-1.491864,2.906265,-0.014139,8.448036,-1.461694,2.940141,0.002912,8.646092,-1.394435,3.007371,-0.002811,9.046057
9,-1.444538,2.920847,-0.007694,8.533041,-1.440238,2.930312,-0.012282,8.588279,-1.433798,2.985636,-0.001630,8.915718



v000_NORMAL_FR6000_FG5955_L000_0,5IN_SENSORC.csv
Signal 4
HOS Features:


,Current_R_kurtosis,Current_R_rms,Current_R_skewness,Current_R_variance,Current_S_kurtosis,Current_S_rms,Current_S_skewness,Current_S_variance,Current_T_kurtosis,Current_T_rms,Current_T_skewness,Current_T_variance
0,-1.440154,1.183351,0.013714,1.400155,-1.462530,1.187334,-0.004084,1.409711,-1.479380,1.189322,-0.009081,1.414460
1,-1.472953,1.181891,-0.005559,1.397140,-1.481570,1.186901,0.010986,1.408946,-1.442576,1.212118,0.003982,1.469461
2,-1.453922,1.189208,0.002142,1.414148,-1.439422,1.190021,-0.009664,1.416253,-1.423441,1.197047,0.010531,1.433197
3,-1.444019,1.187982,0.000902,1.411528,-1.429394,1.189950,0.016354,1.416256,-1.421899,1.200246,0.000512,1.440867
4,-1.404776,1.188887,0.012555,1.413682,-1.428827,1.187659,-0.013043,1.410417,-1.455768,1.194872,0.009562,1.427991
5,-1.441792,1.184604,0.001014,1.403354,-1.470393,1.186745,0.004857,1.408595,-1.477664,1.191722,-0.011004,1.420393
6,-1.482342,1.178776,-0.003632,1.389771,-1.485483,1.189157,0.004345,1.414036,-1.424607,1.201246,0.004784,1.443251
7,-1.465020,1.182686,0.019877,1.398846,-1.432870,1.192201,0.007989,1.421483,-1.441750,1.197475,-0.006852,1.434132
8,-1.439527,1.184099,-0.002507,1.402371,-1.422116,1.189842,-0.000118,1.415931,-1.418499,1.196090,0.007490,1.430848
9,-1.418451,1.181315,0.013562,1.395459,-1.436669,1.189472,-0.003599,1.414840,-1.454980,1.193156,-0.004169,1.423878


### c) Análise exploratória na base criada com os extratores de atributos
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.
- Faça uma análise para cada uma das bases:
    1. Features criada com Fourier
    2. Features criada com HOS
    3. Combinação das duas features (Fourier + HOS). 
> Dica: dê atenção ao coeficiente de correlação.

In [ ]:
profile1 = ProfileReport(joinDataframe(splited_signals[0]), title='signal 0 report', html={'style':{'full_width':True}})
profile2 = ProfileReport(joinDataframe(splited_signals[4]), title='signal 4 report', html={'style':{'full_width':True}})

In [28]:
profile1.to_widgets()

In [31]:
profile2.to_widgets()

In [ ]:
profile1.to_notebook_iframe()

In [ ]:
profile2.to_notebook_iframe()

In [ ]:
È possível notar que 

# Feature Selection

## 1. Realizar um estudo de relevância de atributos utilzando as técnicas exibidas na aula.

Nesse momento utilize a base de dados `Clinical_data_09-09-19-processed.csv`. Uma breve descrição da base:
- Contém um histórico dos registros de pacientes acometidos com uma determinada patologia.
- Os registros são variados, vão desde resposta de questionários médicos de anamnese, por exemplo, "*É fumante nos últimos 5 anos?*", "*Faz uso de álcool constantemente?*", até resultados de exames clínicos como ECG e Ecocardiograma.
- O rótulo dessa base é a coluna `Óbito`.

### Carregue a base

In [ ]:
# code here

### a) Análise exploratória na base de dados
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.

In [ ]:
# code here

### b) Identicar qual a variáveis parecem influenciar no óbito dos pacientes :(
- Utilize o código `feature_selection_framework` https://github.com/navarmn/feature_selection_framework 
- Monte o resultado em um `DataFrame`, as colunas serão referentes aos métodos e as linhas deverão ser os atributos. Coloque 1 quando o método indicar como relevante e 0 quando o método indicar com não relevante;
- Utilize o seguinte critério para validar um atributo como relevante: moda >= 3

In [ ]:
# code here

## 2. QUESTÃO DESAFIO
- Essa etapa não entrará com atividade avaliativa, pois a base de dados a seguir não é tão bonita quanto parece 🤪. Há muitas etapas de pré-processamento, agrupmaento e mais importante de tudo, o usuário tem que fazer as duas tabelas `weekly-infos-before-shrink.csv` e `user-status-after-shrink.csv` conversarem entre si.
- Fique à vontade para tentar e para tirar dúvidas. 😊

Nesse momento utilize a base de dados `weekly-infos-before-shrink.csv` e `user-status-after-shrink.csv`. Uma breve descrição da base:
- Essa base foi retirada de um serviço de streaming de midia. Os atributos são relativos a perfis de consumo de usuários.
- O objetivo é realizar detecção de *churn* (https://resultadosdigitais.com.br/blog/o-que-e-churn/)
- Possui um registro de 17 semanas de uso e os rótulos estáo na tabela `user-status-after-shrink.csv`, que indicam se ao final do período o usuário cancelou e se manteve assinante.

In [ ]:
# code here

### a) Análise exploratória na base de dados
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.
> Exiba a matriz de correlação!

In [ ]:
# code here

### b) Identicar qual a variáveis parecem influenciar no cancelamento da assinatura.

- Utilize o código `feature_selection_framework` https://github.com/navarmn/feature_selection_framework 
- Monte o resultado em um `DataFrame`, as colunas serão referentes aos métodos e as linhas deverão ser os atributos. Coloque 1 quando o método indicar como relevante e 0 quando o método indicar com não relevante;
- Utilize o seguinte critério para validar um atributo como relevante: moda >= 3

In [ ]:
# code here